# 1. ResponseRelevancy / AnswerRelevancy

响应（回答）相关性。输出为0 ～ 1之间，越接近 0 说明效果越差，越接近 1 说明效果越好。


参数：

- name: str, 指标名称,默认为"answer_relevancy"
- question_generation: 根据response(回答)生成问题的提示词。
- strictness: int, 默认值为3。执行 strictness 次，将 question_generation 提示词传递给LLM，得到输出。
- embeddings: BaseRagasEmbeddings, 将 LLM 生成的问题，和真正的问题进行embedding处理，得到对应的向量。
  
  ```python
  from ragas.embeddings import LangchainEmbeddingsWrapper, LlamaIndexEmbeddingsWrapper
  from langchain_ollama import ChatOllama, OllamaEmbeddings
  ```

*******************************************************************
- _required_columns: 需要在数据信息, ["user_input","response"]



## 1.1 question_generation提示词

1. 提示词内容：

"Generate a question for the given answer and Identify if answer is noncommittal. Give noncommittal as 1 if the answer is noncommittal and 0 if the answer is committal. A noncommittal answer is one that is evasive, vague, or ambiguous. For example, "I don't know" or "I'm not sure" are noncommittal answers"

中文翻译:

生成一个针对给定答案的问题，并判断该答案是否为“非承诺性回答”。如果答案是非承诺性的，输出 noncommittal 为 1；如果答案是明确的、有承诺性的，则输出 0。
非承诺性回答指的是那些含糊其辞、模棱两可或逃避问题的回答。例如，“我不知道”或“我不确定”就属于非承诺性回答。

2. 例子：

例子1:

原始句子(user_input)："Where was Albert Einstein born?"

原始句子(respone)："Albert Einstein was born in Germany."

输出：noncommittal = 0

中文翻译：

原始句子(user_input)："阿尔伯特·爱因斯坦出生在哪里？"

原始句子(respone)："阿尔伯特·爱因斯坦出生于德国。"

例子2:

原始句子(user_input)："What was the groundbreaking feature of the smartphone invented in 2023?"

原始句子(respone)："I don't know about the  groundbreaking feature of the smartphone invented in 2023 as am unaware of information beyond 2022."

输出：noncommittal = 1

中文翻译：

原始句子(user_input)："2023 年发明的智能手机的突破性功能是什么？"

原始句子(respone)："我不知道 2023 年发明的智能手机有何突破性功能，因为我不知道 2022 年以后的信息。"


## 1.2 评估过程：
1. 根据响应（response，或者说是"回答"）让LLM提出几个和这个回答相关的问题，并输出答案相对于问题是否为“非承诺性回答”；如a_q1/a_q2/a_q3。是“非承诺性回答”输出"1",否输出"0"。假设输出为 [0, 0, 1]。
    1.  **非承诺性回答**，noncommittal: 指的是那些含糊其辞、模棱两可或逃避问题的回答。
3. 将a_q1/a_q2/a_q3 与 问题(user_input) 进行embedding得到向量，再计算余弦相似度。假设余弦相似度为: [0.8, 0.65, 0.4]
4. 计算 [0.8, 0.65, 0.4].mean() * int(not numpy.any([0, 0, 1])), 得到结果。

In [10]:
from ragas import evaluate, RunConfig
from ragas.metrics import ResponseRelevancy
from langchain_ollama import ChatOllama
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_ollama import ChatOllama, OllamaEmbeddings
from ragas import EvaluationDataset

dataset = EvaluationDataset.from_jsonl("questions_answers_01.txt")

llm = ChatOllama(model="qwen2.5:latest")

metric = ResponseRelevancy(
    # atomicity="low",
    # coverage="high",
    # llm=LangchainLLMWrapper(llm),
    embeddings=LangchainEmbeddingsWrapper(OllamaEmbeddings(model="nomic-embed-text:latest"))
)

result = evaluate(
    dataset=dataset,
    metrics=[metric],
    llm=LangchainLLMWrapper(llm),
    run_config=RunConfig(
        timeout=600,
        max_retries=3,
        max_workers=4
    ),
)
result

Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████| 67/67 [03:44<00:00,  3.36s/it]


{'answer_relevancy': 0.7911}

In [7]:
!ollama list

NAME                            ID              SIZE      MODIFIED     
qwen3:latest                    e4b5fd7f8af0    5.2 GB    6 weeks ago     
bge-large:latest                b3d71c928059    670 MB    6 weeks ago     
dxh_ai/dxh_0.5B:latest          a3c50c65335f    994 MB    2 months ago    
Qwen2.5-0.5B-Instruct:latest    a3c50c65335f    994 MB    2 months ago    
nomic-embed-text:latest         0a109f422b47    274 MB    5 months ago    
qwen2.5:latest                  845dbda0ea48    4.7 GB    6 months ago    
